## Python MySQL
- CRUD Operation

In [2]:
import json
import pymysql

with open ('mysql.json', 'r') as file :
    config_str = file.read()
    
config = json.loads(config_str)

conn = pymysql.connect(
    host = config['host'],
    user = config['user'],
    password = config['password'],
    database = config['database'],
    port = config['port']
)

- CREATE TABLE

In [20]:
sql_user = """
    create table if not exists users (
        uid varchar(20) not null primary key,
        pwd char(44),
        uname varchar(20) not null,
        reg_date datetime default current_timestamp,
        is_deleted int default 0
    );
"""

In [21]:
cur = conn.cursor()
cur.execute(sql_user)

0

- MODIFY TABLE

In [ ]:
cur.execute('alter table users add column email varchar(40)')

- WRITE DATA

In [11]:
cur = conn.cursor()
cur.execute("insert into users(uid, uname) values('admin', '관리자');")

1

In [12]:
cur.execute('select * from users;')
rows = cur.fetchall()
for row in rows :
    print(row)

('admin', None, '관리자', datetime.datetime(2021, 6, 9, 10, 49, 10), 0, None)


In [14]:
sql_insert_multi = """
    insert into users(uid, uname) values
        ('eskim', '김은숙'), ('wjlee', '이우정');
"""

In [ ]:
cur.execute(sql_insert_multi)
conn.commit()   # data flush(실행)

In [20]:
sql_select = """
    select uid, uname, date_format(reg_date, '%y-%m-%d %h:%i') as reg_date, is_deleted, email from users;
"""
cur.execute(sql_select)
rows = cur.fetchall()
for row in rows :
    print(row)

('admin', '관리자', '21-06-09 10:49', 0, None)
('eskim', '김은숙', '21-06-09 10:56', 0, None)
('wjlee', '이우정', '21-06-09 10:56', 0, None)


In [22]:
# 파라메터 이용, placeholder
uid = 'djy'
uname = '대조영'
cur = conn.cursor()
sql = "insert into users(uid, uname) values(%s, %s);"
cur.execute(sql, (uid, uname))
conn.commit()

In [23]:
uid = 'gdhong'
cur = conn.cursor()
sql = "insert into users(uid, uname) values(%s, '홍길동');"
cur.execute(sql, (uid,))
conn.commit()

In [24]:
users = (('jbpark', '박재범'),('vincenzo', '빈센조'))
cur = conn.cursor()
sql = "insert into users(uid, uname) values(%s, %s);"
cur.executemany(sql, users)
conn.commit()

In [ ]:
users = (('jbpark', '박재범'),('vincenzo', '빈센조'))
cur = conn.cursor()
sql = "insert into users(uid, uname) values(%s, %s);"
for user in users :
    cur.execute(sql, user)
conn.commit()

- DELETE DATA

In [25]:
cur = conn.cursor()
sql = "delete from users where uid != 'admin';"
cur.execute(sql)
conn.commit()

In [14]:
import pandas as pd
uids = ['djy', 'eskim', 'gdhong', 'jbpark', 'vincenzo', 'wjlee']
unames = ['대조영', '김은숙', '홍길동', '박재범', '빈센조', '이우정']
df = pd.DataFrame({'uid' : uids, 'uname' : unames})
df

,uid,uname
0,djy,대조영
1,eskim,김은숙
2,gdhong,홍길동
3,jbpark,박재범
4,vincenzo,빈센조
5,wjlee,이우정


- MODIFY DATA

In [ ]:
cur = conn.cursor()
sql = "insert into users(uid, uname) values(%s, %s);"
for i in df.index :
    cur.execute(sql, (df.uid[i], df.uname[i]))
conn.commit()

In [42]:
email = 'admin@korea.com'
uid = 'admin'
cur = conn.cursor()
sql = "update users set email = %s where uid = %s;"
cur.execute(sql, (email, uid))
conn.commit()

In [23]:
cur = conn.cursor()
sql = "update users set email = %s where uid = %s;"
for i in df.index :
    uid = df.uid[i]
    email = f'{uid}@korea.com'  
    cur.execute(sql, (email, uid))
conn.commit()

In [44]:
sql = """
    select uid, uname, email, date_format(reg_date, '%y-%m-%d %h:%i') as reg_date
    from users where is_deleted = 0;
"""
cur = conn.cursor()
cur.execute(sql)
results = cur.fetchall()

In [46]:
uid, uname, email, reg_date = [], [], [], []
for i in results :
    uid.append(i[0])
    uname.append(i[1])
    email.append(i[2])
    reg_date.append(i[3])
df2 = pd.DataFrame({
    'uid' : uid,
    'uname' : uname,
    'email' : email,
    'reg_date' : reg_date
})
df2

,uid,uname,email,reg_date
0,admin,관리자,admin@korea.com,21-06-09 10:49
1,djy,대조영,djy@korea.com,21-06-09 11:33
2,eskim,김은숙,eskim@korea.com,21-06-09 11:33
3,gdhong,홍길동,gdhong@korea.com,21-06-09 11:33
4,jbpark,박재범,jbpark@korea.com,21-06-09 11:33
5,vincenzo,빈센조,vincenzo@korea.com,21-06-09 11:33
6,wjlee,이우정,wjlee@korea.com,21-06-09 11:33


- 데이터 검색

In [47]:
cur = conn.cursor()
cur.execute(sql)
result = cur.fetchone()
result

('admin', '관리자', 'admin@korea.com', '21-06-09 10:49')

In [49]:
result = cur.fetchmany(2)
result

(('gdhong', '홍길동', 'gdhong@korea.com', '21-06-09 11:33'),
 ('jbpark', '박재범', 'jbpark@korea.com', '21-06-09 11:33'))

In [51]:
cur.execute(sql)
result = cur.fetchall()
result

(('admin', '관리자', 'admin@korea.com', '21-06-09 10:49'),
 ('djy', '대조영', 'djy@korea.com', '21-06-09 11:33'),
 ('eskim', '김은숙', 'eskim@korea.com', '21-06-09 11:33'),
 ('gdhong', '홍길동', 'gdhong@korea.com', '21-06-09 11:33'),
 ('jbpark', '박재범', 'jbpark@korea.com', '21-06-09 11:33'),
 ('vincenzo', '빈센조', 'vincenzo@korea.com', '21-06-09 11:33'),
 ('wjlee', '이우정', 'wjlee@korea.com', '21-06-09 11:33'))

In [5]:
keyword = '김'
sql = """
    select uid from users where is_deleted = 0 and uname like %s;
"""

In [29]:
cur = conn.cursor()
cur.execute(sql, keyword + '%')
result = cur.fetchone()
result[0]

'eskim'

In [6]:
cur = conn.cursor()
cur.execute(sql, keyword + '%')
result = cur.fetchall()
result[0][0]

'eskim'

In [7]:
cur.close()
conn.close()